In [1]:
import pandas as pd
import datetime

from lib.ts_backtester import Backtester
from strategies.rsi_2.S_rsi_plot import plot

import pandas_ta as ta
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.chain import ChainId

# Make sure that backtester is defined or not
backtester = Backtester(
    candle_time_bucket=TimeBucket.h4,
    stop_loss_time_bucket=TimeBucket.m1,
    trading_pair=[(ChainId.ethereum, "uniswap-v3", "WETH", "USDC", 0.0005)],
    start_at=datetime.datetime(2021, 1, 1),
    end_at=datetime.datetime(2023, 6, 4),
    reserve_currency="USDC",
)
# try:
#     backtester
# except NameError:
#     print("backtester is not defined")
#     backtester = Backtester(
#         timeframe=TimeBucket.h4,
#         trading_pair=("WBNB", "BUSD"),
#         chain_id=ChainId.bsc,
#         exchange_slug="pancakeswap-v2",
#     )
#     timeframe=TimeBucket.h4,
#     trading_pair=("WBNB", "BUSD"),
#     chain_id=ChainId.bsc,
#     exchange_slug="pancakeswap-v2",
# )

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /home/alpha/.tradingstrategy


In [2]:
ma_long = 200
ma_short = 7
rsi_cutt = 20
atr_distance = 1

import numpy as np


def get_signals(candles):
    close = candles["close"].iloc[-1]
    low = candles["low"].iloc[-1]

    # Calculate indicators
    sma_short = ta.sma(candles["close"], length=ma_short)
    sma_short = ta.sma(candles["close"], length=ma_short).iloc[-1]
    sma_long = ta.sma(candles["close"], length=ma_long).iloc[-1]
    rsi = ta.rsi(candles["close"], length=2).iloc[-1]
    atr = ta.atr(candles["high"], candles["low"], candles["close"], length=14).iloc[-1]

    # Calculate signals
    entry = close >= sma_long and rsi <= rsi_cutt
    exit = close > sma_short
    sl = low - atr * atr_distance
    sl_pct = float(round(sl / candles["open"].iloc[-1], 6))

    indicators = {
        "sma_short": sma_short,
        "sma_long": sma_long,
        "rsi": rsi,
        "atr": atr,
    }
    return entry, exit, sl, sl_pct, indicators


def calculate_size(state, close):
    cash = state.portfolio.get_current_cash()
    return cash * 0.99


def loop(timestamp, universe, state, pricing_model, cycle_debug_data):
    # The pair we are trading
    trades = []
    pair = universe.pairs.get_single()

    candles: pd.DataFrame = universe.candles.get_single_pair_data(
        timestamp, sample_count=ma_long
    )

    if len(candles) < ma_long:
        # Backtest starting.
        # By default get_single_pair_data() returns the candles prior to the `timestamp`,
        # the behavior can be changed with get_single_pair_data(allow_current=True).
        # At the start of the backtest, we do not have any previous candle available yet,
        # so we cannot ask the the close price.
        return trades

    current_price = candles["close"].iloc[-1]

    entry, exit, sl, sl_pct, indicators = get_signals(candles)

    # Create a position manager helper class that allows us easily to create
    # opening/closing trades for different positions
    position_manager = PositionManager(timestamp, universe, state, pricing_model)
    buy_amount = calculate_size(state, current_price)

    if not position_manager.is_any_open():
        if entry:
            # sl = 0.98
            # trades += position_manager.open_1x_long(pair, buy_amount)
            print(sl)
            print(sl_pct)
            # sl = 0.98
            trades += position_manager.open_1x_long(pair, buy_amount, stop_loss_pct=sl_pct)
    else:
        if exit:
            trades += position_manager.close_all()

    plot(state, timestamp, indicators)

    return trades


# start_at = datetime.datetime(2021, 9, 1)
# end_at = datetime.datetime(2021, 9, 7)
start_at = datetime.datetime(2023, 4, 18)
end_at = datetime.datetime(2023, 4, 20)
# end_at = datetime.datetime(2023, 5, 25)



start_at = datetime.datetime(2021, 1, 1)
end_at = datetime.datetime(2023, 9, 7)


backtester.backtest(start_at, end_at, loop)
backtester.stats()
backtester.general_stats()
backtester.plot()

from tradeexecutor.analysis.trade_analyser import build_trade_analysis
from IPython.core.display_functions import display

analysis = build_trade_analysis(backtester.state.portfolio)
from tradeexecutor.analysis.trade_analyser import expand_timeline

timeline = analysis.create_timeline()

expanded_timeline, apply_styles = expand_timeline(
    backtester.universe.universe.exchanges, backtester.universe.universe.pairs, timeline
)

expanded_timeline.drop(
    columns=[
        "Id",
        "Remarks",
        "Exchange",
        "Trade count",
        "Duration",
        "Base asset",
        "Quote asset",
        "PnL %",
        "PnL % raw",
    ],
    inplace=True,
)
expanded_timeline
# expanded_timeline["PnL USD"]

2023-09-07 00:00:00
2059.1764345028255
0.953556
2142.802395044329
0.906246
2433.883243739587
0.933452
2505.7646021137302
0.934673
2928.7179883024387
0.942725
3064.616424813712
0.942216
2996.726892802361
0.953778
3176.0695628317103
0.972196
3044.169928005722
0.961639
3096.3801618917614
0.940439
2902.456470025863
0.934926
2875.607078508663
0.953341
3139.1235602642405
0.958967
3074.4663517017007
0.943262
3139.980542387872
0.944759
3077.2105048588483
0.953591
3111.1549691417713
0.958823
3096.363811286011
0.959239
3807.8795225327067
0.972912
3815.115293245582
0.974371
3574.7083716013935
0.919661
3180.3697165237536
0.849558
3268.3177859996404
0.943212
3410.2816558105615
0.954905
3254.9333979496764
0.944057
3329.591341503022
0.958112
3249.0336246537872
0.939993
3446.811123366813
0.950955
3486.5955474893153
0.974128
3334.245958723512
0.944696
3371.248952458895
0.968065
3339.0041307521433
0.953833
3731.1039114893592
0.97061
3710.972168902557
0.961039
3634.932490086696
0.947106
3945.532928297377

      Opened at Position max value  ... Close mid price USD LP fees
0    2021-07-25          $9,900.00  ...       $2,190.133795  $10.02
1    2021-07-27         $10,136.05  ...       $2,276.549002  $10.24
2    2021-08-03         $10,343.87  ...       $2,502.164046  $10.29
3    2021-08-05         $10,222.74  ...       $2,756.746365  $10.49
4    2021-08-08         $10,745.86  ...       $3,131.915354  $10.93
..          ...                ...  ...                 ...     ...
144  2023-04-17         $14,389.04  ...       $2,101.317871  $14.42
145  2023-04-19         $14,447.56  ...       $1,993.251740  $14.17
146  2023-04-19         $13,886.76  ...       $1,942.486076  $13.75
147  2023-04-21         $13,613.19  ...       $1,844.062899  $13.38
148  2023-05-06         $13,147.08  ...       $1,904.785902  $13.04

[149 rows x 6 columns]